# Hyperparameter tuning for XGBoost
- Grid Search 
    - exhaustive search , evaluates every combinations of hyperparameters for mlmodel 
    - longer time to run when there are a lot of hyper parameteres 
- Random Search 
    - picks fixed set of hyperparameters combinations randomly 
    - sometimes the randomly selected may not include the top performance 
- Bayesian Optimization 
    - gridsearchcv and random search runs on each combination but bayesian optimization is based on previous runs evaluation 

- source 
    - https://colab.research.google.com/drive/18ooFZ4e7cW_zpbvwhBzzhWxCze0Mi6LA#scrollTo=1-FxiavJMirS 

# imports 

In [1]:
from sklearn import datasets 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support as score 

# hyperparameters tuning 
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval

# mlflow
import mlflow 

# dataset creation

In [2]:
data=datasets.load_breast_cancer()
df=pd.DataFrame(data=data.data, columns=data.feature_names)
df['target']=data.target
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# test train split
xtrain,xtest,ytrain,ytest=train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((455, 30), (114, 30), (455,), (114,))

In [4]:
# Standardaize the data
sc=StandardScaler()
xtrain_transformed=pd.DataFrame(sc.fit_transform(xtrain),index=xtrain.index,columns=xtrain.columns)
xtest_transformed=pd.DataFrame(sc.transform(xtest),index=xtest.index,columns=xtest.columns)
# summary after standardization
xtrain_transformed.describe().T 

,count,mean,std,min,25%,50%,75%,max
mean radius,455.0,-1.811494e-15,1.001101,-1.819583,-0.683093,-0.231498,0.459343,3.961679
mean texture,455.0,-3.373126e-15,1.001101,-2.223500,-0.707536,-0.118516,0.563199,4.715674
mean perimeter,455.0,-3.634699e-15,1.001101,-1.809497,-0.690761,-0.242938,0.488480,3.976811
mean area,455.0,-2.537653e-16,1.001101,-1.365036,-0.660205,-0.289597,0.319339,5.208312
mean smoothness,455.0,-4.232024e-15,1.001101,-3.100011,-0.713204,-0.080820,0.633173,4.864642
mean compactness,455.0,1.011157e-15,1.001101,-1.607228,-0.777087,-0.241340,0.528128,3.964311
mean concavity,455.0,9.857804e-16,1.001101,-1.119899,-0.750539,-0.344646,0.547387,4.256736
mean concave points,455.0,5.817081e-16,1.001101,-1.269910,-0.734905,-0.391123,0.673757,4.022271
mean symmetry,455.0,-5.910779e-15,1.001101,-2.345430,-0.701046,-0.069151,0.535429,4.476124
mean fractal dimension,455.0,-3.367270e-15,1.001101,-1.776889,-0.709792,-0.177285,0.464223,4.815921


In [5]:
# summary before standardization
xtrain.describe().T

,count,mean,std,min,25%,50%,75%,max
mean radius,455.0,14.117635,3.535815,7.691000,11.705000,13.300000,15.740000,28.11000
mean texture,455.0,19.185033,4.266005,9.710000,16.170000,18.680000,21.585000,39.28000
mean perimeter,455.0,91.882242,24.322027,47.920000,75.100000,85.980000,103.750000,188.50000
mean area,455.0,654.377582,354.943187,170.400000,420.300000,551.700000,767.600000,2501.00000
mean smoothness,455.0,0.095744,0.013923,0.052630,0.085825,0.094620,0.104550,0.16340
mean compactness,455.0,0.103619,0.052470,0.019380,0.062890,0.090970,0.131300,0.31140
mean concavity,455.0,0.088898,0.079468,0.000000,0.029320,0.061540,0.132350,0.42680
mean concave points,455.0,0.048280,0.038060,0.000000,0.020340,0.033410,0.073895,0.20120
mean symmetry,455.0,0.181099,0.027487,0.116700,0.161850,0.179200,0.195800,0.30400
mean fractal dimension,455.0,0.062757,0.007210,0.049960,0.057645,0.061480,0.066100,0.09744


In [6]:
# params 
params={
    'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1
}

In [7]:
# Step 5 XGBoost Classifier wityh no Hyperparameter tuning 
from  basic import mlflow_utils

experimentid=mlflow_utils.create_mlflow_experiment(experiment_name='XGBoost_with_no_parameters',artifact_location='plain_xgboost',tags={'Xgboost':'no_parameters'})


with mlflow.start_run(run_name='no_parameter_run') as run:
    xgboost=XGBClassifier()
    xgboost=XGBClassifier(seed=0).fit(xtrain_transformed,ytrain)
    xgboost_prediction=xgboost.predict(xtest_transformed)
    # prediction probability 
    xboost_prediction_prob=xgboost.predict_proba(xtest_transformed)[:-1]
    precision,recall,fscore,support=score(ytest,xgboost_prediction)

    #  log it on mlflow
    mlflow.log_params(xgboost.get_params())
    mlflow.log_metrics({'precision':precision[0],'recall':recall[0],'fscore':fscore[0],'support':support[0]})
    mlflow.xgboost.log_model(xgboost,f'{run.info.run_id}-xgboostmodel')
    print(precision,recall,fscore,support)


Experiment XGBoost_with_no_parameters already exists


d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:37:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/12/28 13:37:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[0.95238095 0.95833333] [0.93023256 0.97183099] [0.94117647 0.96503497] [43 71]


# using grid search for XGBoost


In [28]:
from mlflow.models.signature import infer_signature
# define the search grid 
param_grid={
    'colsample_bytree':[0.3,0.5,0.8],
    'reg_alpha':[0,0.5,1,5],
    'reg_lambda':[0,0.5,1,5]
}
#set up score
scoring=['recall']

#Set yp the k-fold cross-validation
kfold=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)

# define grid search 
grid_search=GridSearchCV(
    estimator=xgboost,
    param_grid=param_grid,
    scoring=scoring,
    refit='recall',
    #n_jobs=-1,
    cv=kfold,
    verbose=0
)

#fit grid search
experimentid=mlflow_utils.create_mlflow_experiment(experiment_name='XGBoost_with_no_parameters',artifact_location='plain_xgboost',tags={'Xgboost':'no_parameters'})

with mlflow.start_run(run_name='grid_search_cv') as run:
    grid_results=grid_search.fit(xtrain_transformed,ytrain)
    grid_predict=grid_search.predict(xtest_transformed)

    precision,recall,fscore,support=score(ytest,grid_predict)
    
    #log mlflow results 
    mlflow.log_params(grid_results.best_params_)
    mlflow.log_metrics({'best_score':grid_results.best_score_})
    #mlflow.log_metrics(grid_results.best_params_)
    mlflow.log_metrics({'precision':precision[1],'recall':recall[1],'fscore':fscore[1],'support':support[1]})
    model_signature=infer_signature(xtrain,ytrain,params={'model_name':'model1'})
    mlflow.sklearn.log_model(
        sk_model=grid_results,
        artifact_path='best_model_signature',
        signature=model_signature
    )



Experiment XGBoost_with_no_parameters already exists


d:\anaconda\envs\mlops\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [11]:
# Random Search for XGBoost

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimator

# using random search

In [30]:
param_grid={
    'learning_rate':[0.0001,0.001,0.01,0.1,1],
    'max_depth':range(3,21,3),
    'gamma':[i/10.0 for i in range(0,5)],
    'colsample_bytree':[i/10.0 for i in range(3,10)],
    'reg_alpha':[1e-5,1e-2,0.1,1,10,100],
    'reg_lambda':[1e-5,1e-2,0.1,1,10,100]
}
scoring=['recall']

kfold=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)

In [31]:
random_searh=RandomizedSearchCV(
    estimator=xgboost,
    param_distributions=param_grid,
    n_iter=48,
    scoring=scoring,
    refit='recall',
    # n_jobs=-1,
    cv=kfold,
    verbose=0
    )

with mlflow.start_run(run_name='randomSearchCV') as run:
    random_result=random_searh.fit(xtrain_transformed,ytrain)
    random_predict=random_result.predict(xtest_transformed)
     #log mlflow results 
    mlflow.log_params(random_result.best_params_)
    mlflow.log_metrics({'best_score':random_result.best_score_})
    #mlflow.log_metrics(grid_results.best_params_)
    mlflow.log_metrics({'precision':precision[1],'recall':recall[1],'fscore':fscore[1],'support':support[1]})
    model_signature=infer_signature(xtrain,ytrain,params={'model_name':'model1'})
    mlflow.sklearn.log_model(
        sk_model=random_result,
        artifact_path='best_random_signature',
        signature=model_signature
    )


d:\anaconda\envs\mlops\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


# Bayesian Optimization for XGBoost

In [58]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from typing import Dict, List, Optional 
from functools import partial

def get_classification_metrics(y_true:pd.Series,y_pred:pd.Series,prefix:str)->Dict[str,float]:
    """
    calculate classification metrics
    """
    accuracy=accuracy_score(y_true,y_pred)
    precision=precision_score(y_true,y_pred)
    recall=recall_score(y_true,y_pred)
    f1=f1_score(y_true,y_pred)
    return {
        f'{prefix}_accuracy':accuracy,
        f'{prefix}_precision':precision,
        f'{prefix}_recall':recall,
        f'{prefix}_f1':f1
    }

In [89]:
space={
    'learning_rate':hp.choice('learning_rate',[0.0001,0.001,0.01,0.1,1]),
    'max_depth':hp.choice('max_depth',range(3,21,3)),
    'gamma':hp.choice('gamma',[i/10.0 for i in range(0,5)]),
    'colsample_bytree':hp.choice('colsample_bytree',[i/10.0 for i in range(3,10)]),
    'reg_alpha':hp.choice('reg_alpha',[1e-5,1e-2,0.1,1,10,100]),
    'reg_lambda':hp.choice('reg_lambda',[1e-5,1e-2,0.1,1,10,100])
}

In [90]:
kfold=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)

In [91]:
def objective(params,xtrain_transformed,ytrain,xtest_transformed,ytest):
    #xgboost.set_params(**params)
    with mlflow.start_run(nested=True) as run: 
        xgboost=XGBClassifier(seed=0,**params)
        xgboost.fit(xtrain_transformed,ytrain)
        ypred=xgboost.predict(xtest_transformed)

        metrics=get_classification_metrics(y_true=ytest,y_pred=ypred,prefix='test')
        mlflow.log_metrics(metrics)
        mlflow.log_params(params)
        mlflow.xgboost.log_model(xgboost,f'{run.info.run_id}-model')
        
    return -metrics['test_f1']

In [101]:

with mlflow.start_run(run_name='hyper parameter ') as run:
    trails=Trials()
    best=fmin(
    fn=partial(
        objective,
        xtrain_transformed=xtrain_transformed,
        ytrain=ytrain,
        xtest_transformed=xtest_transformed,
        ytest=ytest
    ),
    space=space,
    algo=tpe.suggest,
    max_evals=48,
    trials=trails
    )
    best_run=sorted(trails.results,key=lambda x:x['loss'])[0]
    print(best_run)
    best_xghoost=XGBClassifier(**best)
    best_xghoost.fit(xtrain_transformed,ytrain)
    ypred=best_xghoost.predict(xtest_transformed)
    metrics=get_classification_metrics(y_true=ytest,y_pred=ypred,prefix='best_test')
    mlflow.log_metrics(metrics)
    mlflow.log_params(best_run)
    mlflow.xgboost.log_model(xgboost,f'{run.info.run_id}-best_model',registered_model_name='best_model')


  0%|          | 0/48 [00:00<?, ?trial/s, best loss=?]

d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:25:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/12/28 17:25:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  2%|▏         | 1/48 [00:04<03:22,  4.30s/trial, best loss: -0.965034965034965]

d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:25:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/12/28 17:25:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  4%|▍         | 2/48 [00:08<03:03,  3.98s/trial, best loss: -0.965034965034965]

...

d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:29:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/12/28 17:29:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 98%|█████████▊| 47/48 [03:37<00:04,  4.56s/trial, best loss: -0.9722222222222222]

d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:29:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/12/28 17:29:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 48/48 [03:42<00:00,  4.63s/trial, best loss: -0.9722222222222222]
{'loss': -0.9722222222222222, 'status': 'ok'}


d:\anaconda\envs\mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:29:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/12/28 17:29:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'best_model'.
Created version '1' of model 'best_model'.


In [100]:
for i in range(3,10):
    print(i/10.0)

0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [88]:
best_xghoost=XGBClassifier(**best)
best_xghoost.fit(xtest_transformed,ytest)

XGBoostError: value 2 for Parameter colsample_bytree exceed bound [0,1]
colsample_bytree: Subsample ratio of columns, resample on each tree construction.